In [ ]:
%config Inline_Backend.figure_format = 'svg'
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob

## Funções auxiliares de manipulação de dados

In [ ]:
def merge_csvs(path="", del_cols=[]):
    """
    Coloca todos os csvs de uma diretoria num único dataframe
    
    path -> string: diretoria dos dados
    del_cols -> [string]: lista de colunas a serem eliminadas dos csvs
    """
    
    bkgd_data, files = [], glob.glob(path + "*.csv")
    for file in tqdm(files, total=len(files), desc="Generating Dataframes"):
        bkgd_data.append(pd.read_csv(file, header=0))
          
    for csv in tqdm(bkgd_data, total=len(bkgd_data), desc="Calculating gen_weights"):
        # Calcular gen_weights
        csv["gen_xsec"] = csv["gen_xsec"].mean()/csv.shape[0]
        csv.rename(columns={"gen_xsec":"gen_weights"}, inplace=True)
        
        # Remover colunas de dados extra
        for col in del_cols:
            if col in list(csv.columns):
                csv.drop(col, axis=1, inplace=True)
             
    # Normalização dos pesos
    csv["gen_weights"] = csv["gen_weights"]/csv["gen_weights"].sum()
                
    bkgd_data = pd.concat(bkgd_data)
    
    return bkgd_data

## Gerar os dataframes e np_arrays dos dados

In [ ]:
# Create dataframe
data_path = "Data/dileptonic/"
del_cols = ["MissingET_Eta", "gen_decay1", "gen_decay2", "gen_sample", "gen_filter", "gen_decay_filter"]
bkgd_data = merge_csvs(data_path, del_cols)

In [ ]:
bkgd_data

In [ ]:
bkgd_data.to_hdf("Bkdg_Data.h5")

## Plots

In [ ]:
bkgd_plot = 

In [ ]:
# Criar os 69 plots
fig, axs = plt.subplots(14, 5, figsize=(40, 60))

for i in range(len(fcnc_data.columns)):
    events, bkgd = fcnc_np[:, i], bkgd_np[:, i]
    row, column = int(i/5), i%5
    
    axs[row, column].set_title(fcnc_data.columns[i])
    axs[row, column].hist(events, label='events', alpha=0.8, bins=50)
    axs[row, column].hist(bkgd, label='bkgd', alpha=0.8, bins=50)
    axs[row, column].set_yscale('log')
    axs[row, column].legend()
    
plt.show()

Significado dos gráficos:
- BTag: Ocorrência ou não de jatos a partir de muões b
- Missing: Momento de partículas não detetadas que falta para se verificar a conservação de momento na colisão
- ScalarHT: Momento total (norma) missing
- Fat_jet_multi: quantos fat jets existem, os plots fat_jets1, 2, etc descrevem cada um

Questões:
- Pico em zero distinto no momento transversal dos fat_jets
- Porque é que o Jet Multi tem frações?
- Fat_Jets têm um Eta pequeno, ocorrem maioritariamente para theta = 90º? Se sim, porquê? 
- Porquê os picos em phi=0? Não deveriam todas as direções transversais ser obter dados aproximadamente iguais?
- Estes dois picos parecem estar associados, no sentido de haver uma direção transversal com muito 'movimento', porquê?

# Obter momentos em x e y

In [ ]:
moment_index_event, moment_index_bkgd = fcnc_data.columns.get_loc("MissingET_MET"), bkgd_data.columns.get_loc("MissingET_MET")
phi_index_event, phi_index_bkgd = fcnc_data.columns.get_loc("MissingET_Phi"), bkgd_data.columns.get_loc("MissingET_Phi")

phi_event, MET_event = fcnc_np[:, phi_index_event], fcnc_np[:, moment_index_event]
phi_bkgd, MET_bkgd = bkgd_np[:, phi_index_event], bkgd_np[:, moment_index_event]

In [ ]:
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 100000

ax = plt.subplot(111, projection='polar')
ax.set_title("Momento transverso em falta", va='bottom')
ax.plot(phi_event, MET_event, label='event')
ax.plot(phi_bkgd, MET_bkgd, label='bkgd')
ax.legend()

plt.show()

In [ ]:
#Obter momentos em x e y
moment_x_event, moment_y_event = MET_event * np.cos(phi_event), MET_event * np.sin(phi_event)
moment_x_bkgd, moment_y_bkgd = MET_bkgd * np.cos(phi_bkgd), MET_bkgd * np.sin(phi_bkgd)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5))

ax1.set_title("Distribuição de momento em falta em x")
ax1.set_yscale('log')
ax1.hist(moment_x_event, label='eventos', bins=50)
ax1.hist(moment_x_bkgd, label='bkgd', bins=50)
ax1.legend()

ax2.set_title("Distribuição de momento em falta em y")
ax2.set_yscale('log')
ax2.hist(moment_y_event, label='eventos', bins=50)
ax2.hist(moment_y_bkgd, label='bkgd', bins=50)
ax2.legend()

plt.show()